In [1]:
import xarray
import numpy as np
import pandas as pd

In [2]:
# ### States
path_LUH2 = 'D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/states.nc'
luh2_states = xarray.open_dataset(path_LUH2, engine="netcdf4", decode_times=False)
luh2_states_worldwide = luh2_states.isel(time=slice(1000, 1173))
luh2_states_worldwide['time'] = pd.date_range(start="1850-01-01", end="2022-01-01", freq='YS')

# ### Transition
transition_LUH2 = 'D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/transitions.nc'
luh2_trans = xarray.open_dataset(transition_LUH2, engine="netcdf4", decode_times=False)
luh2_trans_worldwide = luh2_trans.isel(time=slice(1000, 1173))
luh2_trans_worldwide['time'] = pd.date_range(start='1850-01-01', end='2021-01-01', freq='YS')
luh2_trans_worldwide = luh2_trans_worldwide.drop(['primf_to_urban','secdf_to_urban', 'secdn_to_urban', 'primf_bioh', 'primn_bioh','secmf_bioh','secyf_bioh','secnf_bioh', "urban_to_secdf", "urban_to_secdn","urban_to_c3ann","urban_to_c4ann", "urban_to_c3per",
    "urban_to_c4per", "urban_to_c3nfx", "urban_to_pastr", "c3ann_to_urban", "c4ann_to_urban", "c3per_to_urban", "c4per_to_urban", "c3nfx_to_urban", "pastr_to_urban", "range_to_urban"])

c:\Users\HEFRY ANESTI\AppData\Local\Programs\Python\Python39\lib\site-packages\xarray\coding\times.py:167: SerializationWarning: Ambiguous reference date string: 850-01-01 0:0:0. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0850-01-01 0:0:0). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


In [ ]:
carbon_emission_1971 = xarray.load_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis3_LUC/CARBON_EMISSION-1970-1976_corps.nc", engine="netcdf4")

In [7]:

crop_class = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/FAO/Crop Classification_latest.xlsx", engine="openpyxl", skiprows=1)
crop_class = crop_class.drop('Unnamed: 0', axis=1)
crop_class.rename(columns={'FAO Crops': 'Item'}, inplace=True)


fao_stat = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/FAO/fao_all_data.xlsx", engine="openpyxl")
fao_stat.dropna(axis=0, inplace=True)
fao_stat_yield = fao_stat[fao_stat['Element']=='Yield']
fao_stat_yield.insert(9, 'True Value', fao_stat_yield['Value']*100, True)
faostat_yield = fao_stat_yield.merge(crop_class, on='Item')

In [3]:
grapc = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/GFRAC_GRAPC_Interpolation/GRAPC-RESAMPLE-1970-2100.nc", engine='netcdf4')
grapc_nfbfc = grapc.coords['NFBFC'].data.tolist()

static = 'D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/staticData_quarterdeg.nc'
luh2_static = xarray.open_dataset(static, engine="netcdf4")

country_code = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/ISO-3166-Country-Code.xlsx", engine="openpyxl")
ccode_iso = list(country_code['country-code'])
cname_iso = list(country_code['name'])

ccode_dict = {}
for i, ccode in enumerate(ccode_iso):
    ccode_dict[ccode] = cname_iso[i]
    
ccode_worldwide_int = luh2_static['ccode'].to_numpy().astype('int64')
ccode_convert = np.zeros((720, 1440), dtype="<U64")

for i in range(720):
    for j in range(1440):
        if ccode_worldwide_int[i][j] in ccode_dict.keys():
            ccode_convert[i][j] = ccode_dict[ccode_worldwide_int[i][j]]
        else:
            ccode_convert[i][j] = "Unknown"

ccode_convert_v1 = np.unique(ccode_convert).astype(str)

luh2_trans_worldwide_list = []
for luh2_trans in luh2_trans_worldwide.data_vars:
        luh2_trans_worldwide_list.append(luh2_trans[9:])

In [434]:
# worldwide_intensity = np.zeros((131, 27, 7, 720, 1440), dtype)
grid_intensity = np.zeros((27, 720, 1440), dtype=float)

In [79]:

def grid_check_process(grid_luh, grid_grapc, year, country):

    # kondisi A if: grid 1,1 LUH2 exist = 1 variable & grid 1,1 Grapc exist=1 variable
    if len(grid_luh) == 1 and len(grid_grapc) == 1:
        for grid_grapc in grid_grapc:
            if faostat_yield['IMAGE Classification'] == grid_grapc:
                grid_emisi = carbon_emission_1971[f'to_{faostat_yield['LUH2Class']}'].isel(time=1).sel(lat=-7.125, lon=109.375).values/(grid_grapc * luh2_static['carea'].sel(lat=-7.125, lon=109.375).values)
                grid_intensity = np.array(grid_emisi) 
        return grid_intensity

    # Kondisi B - if: grid 1,1 LUH2 exist = 1 variable & grid 1,1 Grapc exist>1 variable (minimal 2)
    elif len(grid_luh) == 1 and len(grid_grapc) > 1:
        corps =  np.array([(faostat_yield[(faostat_yield['Year'] == year) & 
                                                   (faostat_yield['Area'] == country) &
                                                   (faostat_yield['LUH2Class'] == grid_luh[0][9:])]['IMAGE Classification']),
                            (faostat_yield[(faostat_yield['Year'] == year) & 
                                                    (faostat_yield['Area'] == country) &
                                                    (faostat_yield['LUH2Class'] == grid_luh[0][9:])]['True Value'])])
        
        corps_sum =  np.array([(faostat_yield[(faostat_yield['Year'] == year) & 
                                                    (faostat_yield['Area'] == country) &
                                                    (faostat_yield['LUH2Class'] == grid_luh[0][9:])]['True Value'].sum())])
        for x in range(0, len(corps[0])):
            luas_aktual_grid_grapc = np.array([corps[0][x], corps[1][x], (corps[1][x] * luh2_static['carea'].sel(lat=-7.125, lon=109.375).values), ((corps[1][x] / corps_sum) * 100)]) 
            emisi_intensity = (luas_aktual_grid_grapc[3][0] * carbon_emission_1971['to_c3ann'].isel(time=1).sel(lat=-7.125, lon=109.375).values) / luas_aktual_grid_grapc[1]
            
            print((luas_aktual_grid_grapc[3][0], carbon_emission_1971['to_c3ann'].isel(time=1).sel(lat=-7.125, lon=109.375).values), luas_aktual_grid_grapc[1])
            print(luas_aktual_grid_grapc)

    #Kondisi C
    elif len(grid_luh) > 1 and len(grid_grapc) == 1:
        for items in grid_luh:
            corps = np.array([(faostat_yield[(faostat_yield['IMAGE Classification'] == grid_grapc[0]) &
                                (faostat_yield['Year'] == year) &
                                (faostat_yield['Area'] == country) &
                                (faostat_yield['LUH2Class'] == items[9:])]['Item']),
                                
                                (faostat_yield[(faostat_yield['IMAGE Classification'] == grid_grapc[0]) &
                                (faostat_yield['Year'] == year) &
                                (faostat_yield['Area'] == country) &
                                (faostat_yield['LUH2Class'] == items[9:])]['True Value'])])

            corps_sum = np.array(faostat_yield[(faostat_yield['IMAGE Classification'] == grid_grapc[0]) &
                                (faostat_yield['Year'] == year) &
                                (faostat_yield['Area'] == country) &
                                (faostat_yield['LUH2Class'] == items[9:])]['True Value'].sum())
            
            intensity_corps = []
            if corps.any() > 0:
                for x in range(0, len(corps)): 
                    corps_proportion = np.array([corps[0][x], corps[1][x], ((corps[1][x]/corps_sum)*100)])   

                    intensity_corps.append(carbon_emission_1971['to_c3ann'].isel(time=1).sel(lat=-7.125, lon=109.375).values * corps_proportion[2])
            return intensity_corps
        
    #kondisi D1 D2
    elif len(grid_grapc) > 1 and len(grid_luh) > 1:
        proporsi = list()
        list_mayor_grid = ['Oil, palm fruit', 'Wheat', 'Soybeans', 'Maize', 'Rice']
        for mayor_grid in list_mayor_grid:
                crops = np.array([(faostat_yield[(faostat_yield['IMAGE Classification'] != mayor_grid) &
                                (faostat_yield['Year'] == year) &
                                (faostat_yield['Area'] == country) &
                                (faostat_yield['LUH2Class'] == items[9:])]['IMAGE Classification']),

                                (faostat_yield[(faostat_yield['IMAGE Classification'] != mayor_grid) &
                                (faostat_yield['Year'] == year) &
                                (faostat_yield['Area'] == country) &
                                (faostat_yield['LUH2Class'] == items[9:])]['Item']),
                                
                                (faostat_yield[(faostat_yield['IMAGE Classification'] != mayor_grid) &
                                (faostat_yield['Year'] == year) &
                                (faostat_yield['Area'] == country) &
                                (faostat_yield['LUH2Class'] == items[9:])]['True Value'])])

                crops_sum = np.array(faostat_yield[(faostat_yield['IMAGE Classification'] != mayor_grid) &
                                (faostat_yield['Year'] == year) &
                                (faostat_yield['Area'] == country) &
                                (faostat_yield['LUH2Class'] == items[9:])]['True Value'].sum())

                for x in range(0, len(crops[0])):
                        corps_proportion = np.array([crops[0][x], crops[1][x], crops[2][x], ((crops[2][x]/crops_sum)*100)])
                        proporsi.append(float(corps_proportion[3]))
        return proporsi

### grid jawa tengah

In [80]:
luh2_trans_worldwide['primf_to_c3ann'].isel(time=1).sel(lat=-7.125, lon=109.375).values

grid_gfrac = grapc['GFRAC_res'].isel(time=1).sel(lat=-7.125, lon=109.375).values
atribut = []
for idx, value in enumerate(grid_gfrac):
    if value > 0.0:
        atribut.append(grapc.coords['NGFBFC'].data.tolist()[idx])
        
grid_check_process(grid_luh=['primf_to_c3ann'], grid_gfrac=atribut, year=1970, country='Ghana')

(0.1154804419436513, array(24013.00427246)) 12.0
['Vegetables & fruits' 12.0 9201.6181640625 array([0.11548044])]
(14.723756347815542, array(24013.00427246)) 1530.0
['Pulses' 1530.0 1173206.3159179688 array([14.72375635])]
(5.292853589084019, array(24013.00427246)) 550.0
['Rice' 550.0 421740.83251953125 array([5.29285359])]
(0.08603292924802022, array(24013.00427246)) 8.94
['Plant-based fibers' 8.94 6855.205532226562 array([0.08603293])]
(2.684920275189893, array(24013.00427246)) 279.0
['Other non-food, luxury, spices' 279.0 213937.62231445312
 array([2.68492028])]
(0.3849348064788377, array(24013.00427246)) 40.0
['Vegetables & fruits' 40.0 30672.060546875 array([0.38493481])]
(2.694543645351864, array(24013.00427246)) 280.0
['Vegetables & fruits' 280.0 214704.423828125 array([2.69454365])]
(1.7899468501265954, array(24013.00427246)) 186.0
['Vegetables & fruits' 186.0 142625.08154296875 array([1.78994685])]
(0.15686093364012638, array(24013.00427246)) 16.3
['Other non-food, luxury, spi

C:\Users\HEFRYA~1\AppData\Local\Temp/ipykernel_10028/190737735.py:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  luas_aktual_grid_gfrac = np.array([corps[0][x], corps[1][x], (corps[1][x] * luh2_static['carea'].sel(lat=-7.125, lon=109.375).values), ((corps[1][x] / corps_sum) * 100)])


In [ ]:
for i in range(27):
    if float(luh2_trans_worldwide['secdf_to_c3ann'].isel(time=i).sel(lat=-7.125, lon=109.375).values) > 0.0:
        print(i, luh2_trans_worldwide['secdf_to_c3ann'].isel(time=i).sel(lat=-7.125, lon=109.375).values)


for i in range(27):
    if grapc['GRAPC_res'].isel(time=i).sel(lat=-7.125, lon=109.375).values.any() > 0.0:
        for idx, value in enumerate(grapc['GRAPC_res'].isel(time=i).sel(lat=-7.125, lon=109.375).values):
            if value > 0.0:
                ngfbfc_dfrac = [grapc.coords['NGFBFC'].data.tolist()[idx]]

In [ ]:

worldwide_intensity_arr = np.array()


# output
grid_intensity_netcdf = xarray.Dataset({
        "grid_interp":(["country", 'items', "time", "lat", "lon"], worldwide_intensity_arr)
    },
    coords={
        "country": list(np.unique(ccode_convert)),
        'Items': luh2_trans_worldwide_list,
        "time":pd.date_range("1970-01-01", "2100-01-01", freq='YS'),
        "lon":luh2_states_worldwide.coords["lon"].to_numpy(),
        "lat":luh2_states_worldwide.coords["lat"].to_numpy()
    })


grid_intensity_netcdf.to_netcdf("D:/kerja/asisten riset/vol/milkunarc/cadlan/grid_intensity_netcdf-1970_2100.nc", mode='w', format="NETCDF4")
